In [1]:
# !pip install PyPDF2
# !pip install pyhwp

In [2]:
import os 
import faiss
import numpy as np
import requests
import json
from PyPDF2 import PdfReader
import subprocess
from tqdm import tqdm

In [3]:
# OLLAMA API 앤드포인트
OLLAMA_API = "http://localhost:11434/api"

In [4]:
def get_embedding(text):
    try:
        response = requests.post(f"{OLLAMA_API}/embeddings", json={"model": "EEVE-Korean-10.8B", "prompt": text})
        response.raise_for_status()
        return np.array(response.json()['embedding'])
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

def generate_response(prompt):
    try:
        response = requests.post(f"{OLLAMA_API}/generate", json={"model": "EEVE-Korean-10.8B", "prompt": prompt}, stream=True)
        response.raise_for_status()
        full_response = ""
        for line in response.iter_lines():
            if line:
                decoded_line = line.decode('utf-8')
                try:
                    json_response = json.loads(decoded_line)
                    if 'response' in json_response:
                        full_response += json_response['response']
                except json.JSONDecodeError:
                    print(f"Invalid JSON: {decoded_line}")
        return full_response
    except Exception as e:
        print(f"Error in generate_response: {e}")
        return None

In [5]:
# 문서 준비 및 임베딩 형성 : 파일 형식에 따라 다른 형식으로 데이터를 임베딩 해야함

# PDF 임베딩
def read_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

# HWP 임베딩
def read_hwp(file_path):
    try:
        result = subprocess.run(['hwp5txt', file_path], capture_output=True, text=True)
        return result.stdout
    except Exception as e:
        print(f"Error reading HWP file: {e}")
        return ""

In [6]:
def read_file(file_path):
    _, ext = os.path.splitext(file_path)
    try:
        if ext.lower() == '.pdf':
            return read_pdf(file_path)
        elif ext.lower() == '.hwp':
            return read_hwp(file_path)
        else:
            with open(file_path, 'r', encoding='utf-8', errors='strict') as f:
                return f.read()
    except Exception as e:
        # 파일명과 오류 메시지를 출력합니다.
        print(f"Error processing file: {file_path} - {str(e)}")
        return None

documents = []
embeddings = []
file_list = os.listdir('/Users/dahyun/Desktop/hallym')
for filename in tqdm(file_list, desc="Processing files"):
    file_path = os.path.join('/Users/dahyun/Desktop/hallym', filename)
    text = read_file(file_path)
    if text:
        embedding = get_embedding(text)
        if embedding is not None:
            documents.append(text)
            embeddings.append(embedding)

if not documents or not embeddings:
    print("No documents or embeddings were processed successfully.")
    exit()

Processing files:  12%|███▍                       | 1/8 [00:02<00:18,  2.67s/it]

Error processing file: /Users/dahyun/Desktop/hallym/.DS_Store - 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte


Processing files:  38%|██████████▏                | 3/8 [00:03<00:05,  1.10s/it]

Error processing file: /Users/dahyun/Desktop/hallym/[붙임] 동곡산림문화재단 자기소개서 양식.hwpx - 'utf-8' codec can't decode byte 0x82 in position 14: invalid start byte


Processing files: 100%|███████████████████████████| 8/8 [00:15<00:00,  2.00s/it]


In [7]:
# 임베딩 결과 확인
print(documents)

['\n<표>\n\n<표>\n\n', '체육시설 추첨 신청서\n\n\n<표>\n\n※ 신청자 정보는 추첨 관련 공지를 할 예정이오니, 정확하게 기재하여 주시기 바랍니다.\n', '《 전공·학과, 단과대학 ·스쿨교학팀 전화번호 , 이메일주소 》\n전화번호 (033-248-_______ ), 이메일주소 ( ________@hallym.ac.kr)\n구분 전화 이메일 구분 전화 이메일\n인문\n대학인문대학 교학팀\n(FAX033-248-1505)1500 de1500\n글로벌\n융합대학글로벌융합대학 교학팀\n(FAX033-248-2485)\n1880 de1880인문학부 - - 글로벌학부\n국어국문학전공 1510 de1510 글로벌비즈니스전공\n철학전공 1550 de1550 정보법과학전공\n사학전공 1570 de1570 한중통번역전공\n영어영문학과 1530 de1530 문화산업전공\n중국학과 1590 de1590 한국학전공\n일본학과 1610 de1610 융합과학수사학과 1980 de1980\n통합스쿨통합스쿨 교학팀\n(FAX033-248-3560)3551 de3550 러시아학과 1630 de1630\n사회\n과학\n대학사회과학대학 교학팀\n(FAX033-248-1702)1700 de1700미디어\n스쿨언론방송융합미디어전공3552~4 de1910디지털미디어콘텐츠전공\n심리학과 1720~1 de1720 반도체⋅\n디 스 플 레 이\n스 쿨반도체전공\n3555 de2050사회학과 1740 de1740 디스플레이전공\n사회복지학부1760,\n1769de1760미래융합\n스쿨디지털인문예술전공\n3556~7 de3553사회복지학전공 글로벌협력전공\n노인복지학전공 융합관광경영전공\n정치행정학과 1780 de1780의약신소재전공\n융합신소재공학전공\n광고홍보학과 1949 de1949\n의과대학의과대학 교학팀\n(FAX033-242-2524) 2501 de2500 법학과 1950 de1950\n경제학과 1810 de1810 의예과 , 의학과\n경영\n대학경영대학 교학팀\n(FAX03

In [8]:
# Faiss 인덱스 생성
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

In [9]:
def rag_query(query, k=3):
    query_embedding = get_embedding(query)
    if query_embedding is None:
        return "Failed to generate query embedding."
    
    _, indices = index.search(np.array([query_embedding]), k)
    
    context = "\n".join([documents[i] for i in indices[0]])
    # 프롬프트에 한국말로 결과값 나오도록 설정
    prompt = prompt = f"Context: {context}\n\n질문: {query}\n\n답변을 한국어로 자연스럽게 작성해주세요:"
    
    return generate_response(prompt)

In [11]:
# 사용 예시
query = "체육시설 추첨 관련해서 알려줘"
response = rag_query(query)
print(response)

체육시설 추첨은 제한된 이용 가능한 자원을 공정하게 배분하고, 다양한 사람들에게 시설 사용 기회를 제공하기 위해 설계되었습니다. 이 과정을 통해 신청자들은 주어진 기간 동안 체육시설의 이용을 위한 추첨에 참여할 수 있습니다.

추첨을 위한 신청서를 작성할 때 다음 사항을 주의해야 합니다:

1. 이름과 연락처 정보 기입: 성명(한글과 영문), 전화번호, 이메일 주소와 같은 정확한 개인 정보를 제공하세요. 이는 추후 공지나 의사소통을 위해 필요합니다.
2. 거주지와 직장 정보: 신청서 상에 현재 거주지를 명시하고 해당 체육시설 근처에서 근무하는 경우 직장 정보를 추가로 기재해 주세요. 이것은 신청 순위에 영향을 줄 수 있습니다.
3. 이용 목적 및 선호 시설 선택: 원하는 운동 유형과 관심 있는 구체적인 시설을 명시하세요. 이는 심사 과정에 도움이 됩니다.
4. 이용 기간과 빈도 명시: 체육시설을 이용할 예정인 날짜와 횟수를 알려주시면, 계획에 맞춰 자원을 배분하는 데 유용합니다.
5. 관련 자격 및 경험 기록: 스포츠나 피트니스 분야에서의 관련 자격이나 경험을 언급해 주세요. 이는 평가 과정에서 고려됩니다.
6. 신청서 검토: 기재하신 정보가 정확하고 일관성 있는지 다시 확인하세요. 잘못된 정보는 신청서 처리 지연이나 거부로 이어질 수 있습니다.
7. 지원 서신 첨부(해당되는 경우): 체육시설 추첨에 참여하는 이유를 설명하는 추가 정보를 제공하고자 하는 경우에는 지원 서신을 제출할 수 있습니다. 하지만, 이는 선택 사항이며 귀하의 신청을 결정하는데 직접적인 영향을 주지는 않습니다.
8. 마감일 준수: 체육시설 추첨에 대한 정확한 마감일을 확인하고 기한 내에 신청서를 제출하세요. 늦은 제출된 신청서는 고려되지 않을 수 있습니다.
9. 결과 대기: 추첨 결과는 지정된 공지일에 발표됩니다. 결과가 나오면, 수상 여부를 확인하기 위해 이메일로 또는 전화로 연락받을 예정입니다.
10. 체육시설 사용 규칙 준수: 추첨에 당첨된다면, 해당 시설의 이용 규정을 준수해야 합니

임베딩한 데이터가 제대로 반영되지 않은 이유:
음
1. 임베딩 품질: get_embedding 함수가 제대로 작동하지 않아 모든 문서에 대해 유사한 임베딩을 생성했을 수 있음
2. 인덱스 문제: Faiss 인덱스가 제대로 생성되지 않았거나, 검색이 제대로 작동하지 않을 수 있음
3. 컨텍스트 선택: k=3으로 설정되어 있어, 가장 유사한 3개의 문서만 선택됩니다. 이 값이 너무 작아 관련 문서가 선택되지 않았을 수 있음
4. 모델 응답: OLLAMA 모델이 주어진 컨텍스트를 무시하고 자체 지식에 기반하여 응답했을 수 있음
5. 데이터 처리 오류: 파일 읽기나 전처리 과정에서 오류가 발생해 일부 문서가 제대로 처리되지 않았을 수 있음